<a href="https://colab.research.google.com/github/PiotrSkoupy/PizzaOrderApp/blob/main/RozpoznawanieZUM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!wget  https://github.com/PiotrSkoupy/PizzaOrderApp/blob/main/pizza-audio.zip?raw=true -O  pizza-audio.zip
!unzip pizza-audio
%rm pizza-audio.zip
!wget  https://raw.githubusercontent.com/PiotrSkoupy/PizzaOrderApp/main/pizza-text -O pizza-text

In [2]:
%%capture
!wget https://github.com/danijel3/ASRforNLP/releases/download/v1.0/kaldi.tar.xz
!tar xvf kaldi.tar.xz -C / > /dev/null
%rm kaldi.tar.xz
!for f in $(find /opt/kaldi -name *.so*) ; do ln -sf $f /usr/local/lib/$(basename $f) ; done
!for f in $(find /opt/kaldi/src -not -name *.so* -type f -executable) ; do ln -s $f /usr/local/bin/$(basename $f) ; done
!for f in $(find /opt/kaldi/tools -not -name *.so* -type f -executable) ; do ln -s $f /usr/local/bin/$(basename $f) ; done
!ldconfig
!wget https://raw.githubusercontent.com/danijel3/ASRforNLP/main/lexicon.py
!pip install openfst_python
!pip install arpa
import arpa
import openfst_python as fst
from lexicon import words_to_lexicon

Następnie ściągniemy przykładowe nagrania i zbiór tekstów do trenowania modelu języka.

In [3]:
%%capture
!wget https://github.com/danijel3/ASRforNLP/releases/download/v1.2/models.tar.xz
!tar xvf models.tar.xz > /dev/null
%rm models.tar.xz

In [4]:
%mkdir grammar
%cd grammar
!ln -s ../phonetisaurus
!ln -s ../online
!ln -s ../pizza-audio
%cd /content
%mkdir lm
%cd lm
!ln -s ../phonetisaurus
!ln -s ../online
!ln -s ../pizza-audio
!ln -s ../pizza-text
!ngram-count -order 3 -interpolate -unk -kndiscount -text pizza-text -write-vocab word.list -lm pizza.arpa
%rm pizza.arpa.gz
!gzip pizza.arpa

/content/grammar
/content
/content/lm
rm: cannot remove 'pizza.arpa.gz': No such file or directory


Wygenerujemy przykładowe zdania z tego modelu języka:

### Utworzenie trankrypcji fonetycznej


In [5]:
words=[]
with open('word.list') as f:
  for l in f:
    w=l.strip()

    if w is not None and w!=' ' and w!='-pau-'  and w!='<s>'  and w!='</s>' and len(w)>=3 and w.isalpha()  :
      s= w.strip(')')
      s= w.strip('(')
      s= w.strip('-')
      s= w.strip('<')
      s = w.strip(',')
      s=s.strip('.')
      s=s.strip('?')
      s=s.strip('.);')
      words.append(s)
psyms,wsyms,L=words_to_lexicon(words)
!wc -l word.list

52758 word.list


Tworzenie HCLG.fst

In [6]:
%%capture
L.set_input_symbols(psyms)
L.set_output_symbols(wsyms)
L.write('L.fst')
!grep '#0' phones.txt | cut -f2 -d' ' > wdisambig_phones.int
!grep '#0' words.txt | cut -f2 -d' ' > wdisambig_words.int
!fstaddselfloops wdisambig_phones.int wdisambig_words.int < L.fst | fstarcsort --sort_type=olabel > L_disambig.fst
!gunzip -c pizza.arpa.gz | arpa2fst --disambig-symbol="#0" --read-symbol-table=words.txt - G.fst
!fsttablecompose L_disambig.fst G.fst | fstdeterminizestar --use-log=true | fstminimizeencoded | fstpushspecial > LG.fst
!fstcomposecontext --context-size=2 --central-position=1 --read-disambig-syms=disambig.int --write-disambig-syms=disambig_ilabels.int ilabels LG.fst | fstarcsort --sort_type=ilabel > CLG.fst
!make-h-transducer --disambig-syms-out=disambig_tid.int --transition-scale=1.0 ilabels online/tree online/final.mdl H.fst
!fsttablecompose H.fst CLG.fst - | fstdeterminizestar --use-log=true - - | fstrmsymbols disambig_tid.int - - | fstminimizeencoded - - | add-self-loops --self-loop-scale=0.1 --reorder=true online/final.mdl - HCLG.fst

### Rozpoznawanie mowy

Tworzymy listę plików `wav.scp` i `spk2utt` z katalogu `pizza-audio`:

In [7]:
%%bash
for f in pizza-audio/*.wav ; do echo $(basename $f .wav) $(readlink -f $f) ; done > wav.scp
cut -f1 -d' ' wav.scp | awk '{print $1, $1}' > spk2utt

Rozpoznajemy dzwieki z katalogu pizza-audio

In [9]:
!online2-wav-nnet3-latgen-faster --word-symbol-table=words.txt --config=online/conf/online.conf online/final.mdl HCLG.fst ark:spk2utt scp:wav.scp ark:lat

online2-wav-nnet3-latgen-faster --word-symbol-table=words.txt --config=online/conf/online.conf online/final.mdl HCLG.fst ark:spk2utt scp:wav.scp ark:lat 
LOG (online2-wav-nnet3-latgen-faster[5.5.971~1-07043]:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (online2-wav-nnet3-latgen-faster[5.5.971~1-07043]:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (online2-wav-nnet3-latgen-faster[5.5.971~1-07043]:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (online2-wav-nnet3-latgen-faster[5.5.971~1-07043]:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (online2-wav-nnet3-latgen-faster[5.5.971~1-07043]:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (online2-wav-nnet3-latgen-faster[5.5.971~1-07043]:CompileLooped():nnet-compile-looped.cc:345) Spent 0.00840902 seconds in looped compilation.
Plik1_ZUM10 nie nia chca 
LOG (online2-wav-nnet3-latgen-faster[5.5.971~1-07043]:main